<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/actividades%20extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
!pip install pyxlsb

In [ ]:
#designar nomes de ficheiros
template_acordos_pagamento="042024_TEMPLATE_ACORDOS_PAGAMENTO_v3.xlsx"
template_pagamento="042024_TEMPLATE_PAGAMENTO.xlsx"

erro_template_acordos_pagamento="Erro_acordos_pagamento.xlsx"

In [ ]:
#upload dos ficheiros "template acordos pagamento" e "template pagamento"
from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()


In [ ]:
#ler a primeira folha (original) do "templaye_acordos_pagamento"

acordos_pagamento=pd.read_excel(template_acordos_pagamento, sheet_name=0)

In [ ]:
#visualização da dataframe "acordos_pagamento"
acordos_pagamento.head()

In [ ]:
#formatar a coluna "Payment Agreement Date" para retirar a hora

acordos_pagamento["Payment Agreement Date"]=acordos_pagamento.iloc[:, -1].apply(lambda i: i.split(" ")[0])

acordos_pagamento.head()

In [ ]:
#eliminar duplicados

acordos_pagamento2=acordos_pagamento.copy()

print(acordos_pagamento2.shape)

acordos_pagamento2.drop_duplicates(ignore_index=True, inplace=True)

print(acordos_pagamento2.shape)

acordos_pagamento2.head()

In [ ]:
#filtrar acordos de pagamento por "em curso" (na coluna status) e substituir "em curso" por "aprovado"

acordos_aprovado=acordos_pagamento2[acordos_pagamento2["Status"].str.contains("em curso", case=False)]

acordos_aprovado.loc[acordos_aprovado["Status"].str.contains("em curso", case=False), "Status 2"]="Aprovado"

acordos_aprovado.drop("Status", axis=1, inplace=True)

acordos_aprovado=acordos_aprovado.reindex(["Payment Agreement ID", "Service Request ID", "Status 2", "Payment Agreement Date"], axis=1)

acordos_aprovado.columns=["Payment Agreement ID", "Service Request ID", "Status", "Payment Agreement Date"]

print(acordos_aprovado.shape)

acordos_aprovado.head()

In [ ]:
#fazer upload do ficheiro de erro (SRs de acordos de pagamento que não existem em RI)
from google.colab import files
uploaded3 = files.upload()


In [ ]:
#ler ficheiro erro_acordos_pagamento
erro_acordos_pagamento=pd.read_excel(erro_template_acordos_pagamento, sheet_name=0)

In [ ]:
#visualizar ficheiro erro_acordos_pagamento
erro_acordos_pagamento.iloc[:, :5].head()

In [ ]:
# formatar ficheiro erro_acordos_pagamento
erro_acordos_pagamento=erro_acordos_pagamento.iloc[2:, :5]

erro_acordos_pagamento.columns=["Payment Agreement ID", "Service Request ID", "Status", "Payment Agreement Date", "RI Info"]

erro_acordos_pagamento.iloc[:, -2]=erro_acordos_pagamento.iloc[:,-2].apply(lambda i: i.strftime("%Y/%m/%d"))

erro_acordos_pagamento.index=range(erro_acordos_pagamento.iloc[:, 0].count())

print(erro_acordos_pagamento.shape)
erro_acordos_pagamento.head()

In [ ]:
#eliminar linhas na dataframe "acordos_aprovado" cujas SRs não estão em RI (cruzar com info da dataframe "erro_acordos_pagamento")

#criar colunas "chave" em "acordos_aprovado" e "erro_acordos_pagamento"

acordos_aprovado["chave"]=acordos_aprovado["Payment Agreement ID"].astype("str") + acordos_aprovado["Service Request ID"].astype("str")

erro_acordos_pagamento["chave"]=\
erro_acordos_pagamento["Payment Agreement ID"].astype("str") + erro_acordos_pagamento["Service Request ID"].astype("str")




In [ ]:
#confirmar se todas as chaves de "acordos_aprovado" estão em "erro_acordos_pagamento"
print(acordos_aprovado.loc[~acordos_aprovado["chave"].isin(erro_acordos_pagamento["chave"]), "chave"].count())

In [ ]:
#eliminar linhas na dataframe "acordos_aprovado" cujas SRs não estão em RI (cruzar com info da dataframe "erro_acordos_pagamento")


#quantas SRS não foram encontradas em RI:

print(erro_acordos_pagamento.loc[erro_acordos_pagamento.iloc[:, -2].isna()==False,"RI Info"].count())

#lista_erro inclui as linhas de SRs que não existem em RI
lista_erro=erro_acordos_pagamento.loc[erro_acordos_pagamento.iloc[:, -2].isna()==False, "chave"].tolist()

acordos_aprovado2=acordos_aprovado[~acordos_aprovado["chave"].isin(lista_erro)]

#apagar coluna "chave" em "acordos_aprovado2"

acordos_aprovado2.drop("chave", axis=1, inplace=True)

print(acordos_aprovado2.shape)

acordos_aprovado2.head()


In [ ]:
#eliminar eventuais registos duplicados nas colunas "Payment Agreement ID", "Service Request ID", "Status", "Payment Agreement Date" - *TESTAR

print(acordos_aprovado2.shape)

acordos_aprovado2["quant"]=acordos_aprovado2.apply(lambda i: "".join(i.values.astype("str")), axis=1)

if acordos_aprovado2["quant"].duplicated().sum()>0:
  indice=[]
  indice=acordos_aprovado2[acordos_aprovado2["quant"].duplicated()].index
  acordos_aprovado2.drop(index=indice, inplace=True)

#eliminar coluna "quant"
acordos_aprovado2.drop("quant", axis=1, inplace=True)


print(acordos_aprovado2.shape)

In [ ]:
#formatar coluna com data type e ordenar tabela por "Payment Agreement ID" e "Payment Agreement Date"

acordos_aprovado2["Payment Agreement Date"]=acordos_aprovado2["Payment Agreement Date"].apply(lambda i: datetime.strptime(i, "%d/%m/%Y").date())


acordos_aprovado3=acordos_aprovado2.sort_values(["Payment Agreement ID", "Payment Agreement Date"], ascending=[True, False], ignore_index=True)

In [ ]:
#criar colunas "contagem" e "índice"


def contagem (db, col):
  #criar lista para coluna "contagem"
  listaa=list(map(lambda i: db.loc[db[col]==i, col].count(), db[col].tolist()))
  return listaa


def indice (db, col):
  indice=[]
  for i in db.groupby(col) [col].count().tolist():
    for j in range(1, i+1):
      indice.append(j)
  return indice

acordos_aprovado3["contagem"]=contagem(acordos_aprovado3, "Payment Agreement ID")

acordos_aprovado3["indice"]=indice(acordos_aprovado3, "Payment Agreement ID")




In [ ]:
#ver acordos_aprovado3

print(acordos_aprovado3.shape)


acordos_aprovado3.head()

In [ ]:
#filtrar linhas com índice igual a 1 e remover colunas "contagem" e "índice"

acordos_aprovado4=acordos_aprovado3[acordos_aprovado3["indice"]==1]
acordos_aprovado4.drop(["contagem", "indice"], axis=1, inplace=True)
print(acordos_aprovado4.shape)

acordos_aprovado4.head()

In [ ]:
#ordenar "acordos_aprovado4" por "Service Request ID" (ordem ascendente)

acordos_aprovado5=acordos_aprovado4.sort_values(by=["Service Request ID", "Payment Agreement Date"], ascending=[True, False])

acordos_aprovado5.head()

In [ ]:
#criar coluna "contagem" e "índice" na tabela "acordos_aprovado5"

acordos_aprovado5["contagem"]=contagem(acordos_aprovado5, "Service Request ID")

acordos_aprovado5["indice"]=indice(acordos_aprovado5, "Service Request ID")
print(acordos_aprovado5.shape)
acordos_aprovado5.head()

In [ ]:
#filtrar linhas com índice igual a 1

acordos_aprovado6=acordos_aprovado5[acordos_aprovado5["indice"]==1]
print(acordos_aprovado6.shape)

In [ ]:
#recolocar coluna "Payment Agreement Date" como texto
acordos_aprovado6["Payment Agreement Date"]=acordos_aprovado5["Payment Agreement Date"].apply(lambda i: i.strftime("%d/%m/%Y"))

acordos_aprovado6.head()

In [ ]:
#exportar tabela "acordos_aprovado5" e fazer download

acordos_aprovado6.to_excel("acordos de pagamento.xlsx", index=False)

files.download("acordos de pagamento.xlsx")